<a href="https://colab.research.google.com/github/ShraddhaSharma24/Natural-Language-Processing/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lakshmi25npathi_imdb_dataset_of_50k_movie_reviews_path = kagglehub.dataset_download('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
df=data.iloc[: 10000]

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.drop_duplicates(inplace=True)

/tmp/ipykernel_30/3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<,*?>'),'',raw_text)
    return cleaned_text

In [ ]:
df['review']=df['review'].apply(remove_tags)

/tmp/ipykernel_30/2928534252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_tags)


In [ ]:
df['review']=df['review'].apply(lambda x:x.lower())

/tmp/ipykernel_30/2697927567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:x.lower())


In [ ]:
from nltk.corpus import stopwords
sw_list=stopwords.words('english')
df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:"".join(x))

/tmp/ipykernel_30/3253563264.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:"".join(x))


In [ ]:
df['review']

0       onereviewersmentionedwatching1ozepisodehooked....
1       wonderfullittleproduction.<br/><br/>thefilming...
2       thoughtwonderfulwayspendtimehotsummerweekend,s...
3       basicallythere'sfamilylittleboy(jake)thinksthe...
4       pettermattei's"lovetimemoney"visuallystunningf...
                              ...                        
9995    fun,entertainingmoviewwiigermanspy(julieandrew...
9996    givebreak.anyonesay"goodhockeymovie"?knowmovie...
9997    moviebadmovie.watchingendlessseriesbadhorrormo...
9998    movieprobablymadeentertainmiddleschool,earlyhi...
9999    smashingfilmfilm-making.showsintensestrangerel...
Name: review, Length: 9983, dtype: object

In [ ]:
import gensim

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
story=[]
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [ ]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(441780, 1004425)

In [ ]:
len(model.wv.index_to_key)

13309

In [ ]:
def document_vector(doc):
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
def document_vector(doc):
    # Filter words present in the model vocabulary
    doc = [word for word in doc.split() if word in model.wv.index_to_key]

    # Handle empty documents
    if not doc:
        return np.zeros(model.vector_size)  # Return a zero vector of appropriate size

    # Get the vectors for each word and calculate the mean
    vectors = [model.wv[word] for word in doc]
    return np.mean(vectors, axis=0)

In [ ]:
document_vector(df['review'].values[0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
from tqdm import tqdm

In [ ]:
X=[]
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [00:02<00:00, 3851.45it/s]


In [ ]:
X=np.array(X)

In [ ]:
X.shape

(9983, 100)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

y=encoder.fit_transform(df['sentiment'])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
mnb=GaussianNB()
mnb.fit(X_train,y_train)
y_pred=mnb.predict(X_test)
accuracy_score(y_test,y_pred)

/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


0.48247078464106846

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.48247078464106846